# Deploying MCP Server to AWS Bedrock Agent Core Runtime

In this notebook, we'll deploy our restaurant booking MCP server to **AWS Bedrock Agent Core Runtime** - a serverless platform designed for AI agents and tools.

## What is AWS Bedrock Agent Core Runtime?

Agent Core Runtime provides:
- **Serverless deployment** - No infrastructure management
- **Secure isolation** - Each session runs in its own environment
- **MCP protocol support** - Native support for Model Context Protocol
- **Auto-scaling** - Handles traffic spikes automatically
- **Framework agnostic** - Works with any agent framework

Learn more: [Agent Core Documentation](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agents-tools-runtime.html)

## Step 1: Load Table Name from Previous Notebook

In [ ]:
%store -r table_name

In [ ]:
print(f"Using DynamoDB table: {table_name}")

## Step 2: Examine Our Enhanced MCP Server

Let's look at our revised MCP file that we'll be deploying to Agent Core Runtime. This version includes an `update_booking` method to update existing bookings.

In [ ]:
# Display the key components of our MCP server
with open('restaurant_mcp_agentcore_rt.py', 'r') as f:
    lines = f.readlines()
    
print("Key components of our MCP server:")
print("1. BedrockAgentCoreApp integration")
print("2. Strands agent framework")
print("3. Amazon Nova Pro model")
print("4. DynamoDB operations (create, read, update, delete bookings)")
print("5. Structured logging for production")
print("\nTotal lines of code:", len(lines))

## Step 3: Create Agent Core Execution Role

Before deploying, we need to create an **execution role** that grants our agent permissions to:
- Access DynamoDB tables
- Invoke Bedrock models  
- Write logs to CloudWatch
- Access ECR repositories

In [ ]:
from utils import create_agent_core_execution_role

# Create execution role for our restaurant booking agent
agent_core_role_arn = create_agent_core_execution_role(
    agent_name='restaurant_booking', 
    dynamodb_table_name=table_name
)

print(f"Agent Core Role ARN: {agent_core_role_arn}")

# Store for use in subsequent cells
%store agent_core_role_arn

## Step 4: Configure Agent Core Deployment

Now we'll configure our deployment using the Agent Core CLI. This will:
1. Create configuration files (`.bedrock_agentcore.yaml` and `Dockerfile`)
2. Set up ECR repository for container storage
3. Configure the execution role and dependencies

**Note**: Replace `<ROLE_ARN>` below with the actual role ARN from the previous step.

In [ ]:
# Configure Agent Core deployment
# You need to replace the placeholder with the actual role ARN
import subprocess

cmd = f'''agentcore configure -e restaurant_mcp_agentcore_rt.py \
    --execution-role {agent_core_role_arn} \
    --requirements-file "requirements.txt" \
    --authorizer-config null \
    --ecr "auto"'''

print("Running configuration command...")
result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("Errors:", result.stderr)

## Step 5: Deploy to Agent Core Runtime

Now we'll deploy our agent to the cloud! This uses **CodeBuild** for cloud-based container building (no local Docker required).

The deployment process:
1. **Package** application with dependencies
2. **Build** container image using CodeBuild
3. **Push** to ECR repository
4. **Deploy** to Agent Core Runtime
5. **Create** runtime endpoints

In [ ]:
# Deploy to Agent Core Runtime using CodeBuild
!agentcore launch

## Step 6: Test Our Deployed Agent

🎉 **Success!** Your MCP server is now running in the cloud. Let's test it by making a restaurant booking.

In [ ]:
# Test creating a new booking
!agentcore invoke '{"prompt": "can i make a booking for tonight, for 6 people, at 7 PM? Name is Ferdinand."}'

**Expected Result**: You should receive a booking confirmation with a unique booking ID.

## Step 7: Test Booking Retrieval

Let's verify we can retrieve booking details:

In [ ]:
# Test retrieving booking details 
# Replace the booking ID with the one from your previous booking
!agentcore invoke '{"prompt": "can you tell me the booking details for booking id 999999."}'

## Step 8: Test Booking Updates

Now let's test our new `update_booking` functionality:

In [ ]:
# Test updating a booking to accommodate more people
!agentcore invoke '{"prompt": "can i update a booking i made? Its under Ferdinand. I need it for 10 people now."}'

## Step 9: Monitor Your Deployment

Check the status of your deployment and get information about logs:

In [ ]:
# Check deployment status and get monitoring information
!agentcore status

### Understanding the Status Output:

The status shows:
- **Agent Info**: Name, ID, ARN, timestamps
- **Configuration**: Region, account, execution role, ECR repository
- **Endpoint Status**: Runtime endpoint status and availability
- **Logs**: CloudWatch log locations for debugging

### Viewing Logs:
```bash
aws logs tail /aws/bedrock-agentcore/runtimes/YOUR-AGENT-ID --follow
```

## Step 10: Clean Up Resources

⚠️ **Important**: Clean up resources to avoid unnecessary charges. This will permanently delete your agent deployment.

First, get your agent runtime ID from the status output above, then run:

In [ ]:
import boto3

# Initialize the Agent Core Control client
agent_rt_client = boto3.client('bedrock-agentcore-control')

# Replace 'YOUR-AGENT-RUNTIME-ID' with the actual ID from 'agentcore status'
# Example: 'restaurant_mcp_agentcore_rt-lNB6ib7IOm'

agent_runtime_id = 'YOUR-AGENT-RUNTIME-ID'  # UPDATE THIS

try:
    response = agent_rt_client.delete_agent_runtime(
        agentRuntimeId=agent_runtime_id
    )
    print(f"Deletion initiated. Status: {response['status']}")
    print(f"Full response: {response}")
except Exception as e:
    print(f"Error deleting agent runtime: {e}")
    print("Make sure to replace 'YOUR-AGENT-RUNTIME-ID' with the actual ID from 'agentcore status'")

### Verify Deletion

Check the status to confirm deletion is in progress:

In [ ]:
# Verify deletion status
!agentcore status

**Expected Result**: The endpoint status should show `DELETING` initially, then `Unknown` when deletion is complete (takes a few minutes).

## 🎯 Summary

Congratulations! You have successfully:

✅ **Deployed an MCP Server** to AWS Bedrock Agent Core Runtime  
✅ **Configured Security** with proper IAM roles and permissions  
✅ **Tested Cloud Deployment** with booking operations  
✅ **Monitored Your Agent** using status and logging capabilities  
✅ **Cleaned Up Resources** to avoid unnecessary costs  

## 🔑 Key Learnings

### Agent Core Runtime Benefits:
- **Serverless**: No infrastructure management
- **Secure**: Isolated execution environments
- **Scalable**: Auto-scaling based on demand
- **Protocol Support**: Native MCP integration

### Deployment Process:
1. **Configure**: Set parameters and security
2. **Build**: Create containers (CodeBuild recommended)
3. **Deploy**: Push to runtime with auto-scaling
4. **Monitor**: Track performance via CloudWatch

## 📚 Additional Resources

- **[Agent Core Developer Guide](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/)**: Complete documentation
- **[MCP Deployment Guide](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-mcp.html)**: MCP-specific deployment info
- **[Runtime Architecture](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-how-it-works.html)**: Understanding the platform

## 🚀 Next Steps

Consider these next steps for your learning journey:

### 1. **Enhance Your Agent**
- Add availability checking logic
- Implement multiple restaurant locations
- Add customer preferences handling

### 2. **Production Features**
- Set up CI/CD pipelines
- Implement monitoring and alerting
- Configure auto-scaling policies

### 3. **Explore Agent Core Ecosystem**
- Try Agent Core Gateway for API integration
- Experiment with Agent Core Browser for web automation
- Build multi-agent systems with agent-to-agent communication

### 4. **Advanced Patterns**
- Implement streaming responses
- Add memory capabilities
- Create complex workflow orchestration

Great work on completing this deployment workshop! 🎉